**Adding Trained Model to Hopsworks Registry: A Step towards Streamlined Model Deployment and Management**

In this notebook, the final trained model that is stored as an artifact in Weights and Biases (and locally too), is added to the Hopsworks model registry.

In [1]:
# import libraries
import pandas as pd
import hopsworks
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

In [2]:
# following is required for defining schema
# get train set
train_df = pd.read_csv(r"E:\NYC Taxi Trip Duration Prediction\data\training_datasets\train.csv") # load train.csv
print(f'Shape (Training): {train_df.shape}') 

# convert columns to categorical as per train sets
train_df = train_df.astype({
    'vendor_id': 'category', 
    'passenger_count': 'category',
    'store_and_fwd_flag': 'category',
    '2AMto7AM_7AMto2AM': 'category',
    'holiday': 'category'
})

# split training data set: features and target (also removed columns not used for training)
X_train = train_df[train_df.columns.difference([
    'id', 
    'pickup_datetime', 
    'trip_duration', 
    'hour',
    'minute',
    'store_and_fwd_flag'])]
y_train = train_df["trip_duration"] # target
print(f'Train Shape: {X_train.shape}, {y_train.shape}')

Shape (Training): (1216082, 20)
Train Shape: (1216082, 14), (1216082,)


In [3]:
# login to hopsworks
try:
    project = hopsworks.login(
        project="nyc_taxi_trip_duration", 
        api_key_file=r'E:\NYC Taxi Trip Duration Prediction\config\hopsworks_api_key'
    )
except hopsworks.exceptions.HopsworksRestAPIError as e:
    print(f"Unable to login to Hopsworks: {e}")

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/37032


In [4]:
# get Hopsworks Model Registry handle
mr = project.get_model_registry()

# save to hopsworks model registry
input_example = X_train.sample() # instance of a valid model input
input_schema = Schema(X_train) # describes type and shape of inputs for your model
output_schema = Schema(y_train) # describes type and shape of outputs (predictions) for your model
model_schema = ModelSchema(input_schema, output_schema)

# register model to hopsworks registry
final_xgboost_model = mr.python.create_model(
    version=1,
    name="final_xgboost", 
    metrics={"train_rmsle":0.351, "test_rmsle":0.439},
    model_schema=model_schema,
    input_example=input_example, 
    description="Final model trained using XGBoost Regressor on Jan-May train data and June test data.")

final_xgboost_model.save(r'E:\NYC Taxi Trip Duration Prediction\model\final_xgb_model.bin')

Connected. Call `.close()` to terminate connection gracefully.


  0%|          | 0/6 [00:00<?, ?it/s]

Model created, explore it at https://c.app.hopsworks.ai:443/p/37032/models/final_xgboost/1


Model(name: 'final_xgboost', version: 1)